In [1]:
#importing the libraries
import torch
import numpy as np
import matplotlib.pyplot as plt

In [3]:
#importing the dataset
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()
x = data['data']
y = data['target']
print("shape of x: {}\nshape of y: {}".format(x.shape,y.shape))


shape of x: (569, 30)
shape of y: (569,)


In [4]:
#feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x = sc.fit_transform(x)

In [5]:
#defining dataset class
from torch.utils.data import Dataset, DataLoader
class dataset(Dataset):
    def __init__(self,x,y):
        self.x = torch.tensor(x,dtype=torch.float32)
        self.y = torch.tensor(y,dtype=torch.float32)
        self.length = self.x.shape[0]

    def __getitem__(self,idx):
        return self.x[idx],self.y[idx]
    def __len__(self):
        return self.length
trainset = dataset(x,y)
#DataLoader
trainloader = DataLoader(trainset,batch_size=64,shuffle=False)

In [10]:
#defining the network
from torch import nn
from torch.nn import functional as F
class Net(nn.Module):
    def __init__(self,input_shape):
        super(Net,self).__init__()
        self.network = nn.Sequential(
        nn.Linear(input_shape,32),
        nn.ReLU(),
        nn.Linear(32,64),
        nn.ReLU(),
        nn.Linear(64,1),
        nn.Sigmoid()
        )
        
    def forward(self,x):
        x = self.network(x)
        return x
#         self.fc1 = nn.Linear(input_shape,32)
#         self.fc2 = nn.Linear(32,64)
#         self.fc3 = nn.Linear(64,1)
#     def forward(self,x):
#         x = torch.relu(self.fc1(x))
#         x = torch.relu(self.fc2(x))
#         x = torch.sigmoid(self.fc3(x))
#         return x

In [11]:
#hyper parameters
learning_rate = 0.01
epochs = 700
# Model , Optimizer, Loss
model = Net(input_shape=x.shape[1])
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)
loss_fn = nn.BCELoss()

In [12]:
#forward loop
losses = []
accur = []
for i in range(epochs):
  for j,(x_train,y_train) in enumerate(trainloader):
    
    #calculate output
    output = model(x_train)
 
    #calculate loss
    loss = loss_fn(output,y_train.reshape(-1,1))
 
    #accuracy
    predicted = model(torch.tensor(x,dtype=torch.float32))
    acc = (predicted.reshape(-1).detach().numpy().round() == y).mean()
    #backprop
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  if i%50 == 0:
    losses.append(loss)
    accur.append(acc)
    print("epoch {}\tloss : {}\t accuracy : {}".format(i,loss,acc))

epoch 0	loss : 0.7095210552215576	 accuracy : 0.3110720562390158
epoch 50	loss : 0.17297674715518951	 accuracy : 0.9490333919156415
epoch 100	loss : 0.0935879573225975	 accuracy : 0.9736379613356766
epoch 150	loss : 0.08016348630189896	 accuracy : 0.9806678383128296
epoch 200	loss : 0.07392395287752151	 accuracy : 0.9859402460456942
epoch 250	loss : 0.0680769607424736	 accuracy : 0.9859402460456942
epoch 300	loss : 0.06141757592558861	 accuracy : 0.9859402460456942
epoch 350	loss : 0.05601200461387634	 accuracy : 0.9876977152899824
epoch 400	loss : 0.05150778964161873	 accuracy : 0.9876977152899824
epoch 450	loss : 0.04761691391468048	 accuracy : 0.9876977152899824
epoch 500	loss : 0.04411093518137932	 accuracy : 0.9876977152899824
epoch 550	loss : 0.04170794412493706	 accuracy : 0.9929701230228472
epoch 600	loss : 0.039679065346717834	 accuracy : 0.9929701230228472
epoch 650	loss : 0.03745563328266144	 accuracy : 0.9929701230228472
